In [ ]:
#@title Funcionnes:  Se debe agregar un archivo con el nombre pedidos.xlsx en donde las columnas importantes son "FTO" para los formatos, "Saldo" para los KG a producir y "IV" con los IV de los clientes.


!pip install pulp
import pandas as pd
from pulp import *
import operator
from itertools import chain
from collections import defaultdict

def ordenamiento(ordenes, peso, b, d, cliente ):        
        diccionario = {}
        m = sum(d)
        bobinas0 = []
        kilos0 = []
        cortes0 = []
        kiloporbob0 = []
        kilosprod0 = []
        valores0 = []      
        for i in range(len(cliente)):
            diccionario[cliente[i]] = b[i],d[i]
            
        for i in range(len(d)):
            x = d[i]/((b[i]*(peso/2))/383)
            y = d[i]
            z = b[i]
            k = ((b[i]*(peso/2))/383)
            bobinas0.append(round(x))
            kilos0.append(y)
            cortes0.append(round(z))
            kiloporbob0.append(round(k))
            
        for i in range(len(cortes0)):
            f = kiloporbob0[i] * bobinas0[i]
            kilosprod0.append(f)
            
        for i in range(len(cortes0)):
            a = (kilos0[i], kilosprod0[i], bobinas0[i], kiloporbob0[i])
            valores0.append(a) 

        dictutil0 ={"IV":cliente,"formato":cortes0, "Kg pedidos": kilos0, "kg programados":kilosprod0, "bobinas": bobinas0}
        dfutil0 = pd.DataFrame(dictutil0)
       
        #pedidos = dict(zip(cortes,bobinas))
        df_pedidos = dfutil0.loc[dfutil0.index.repeat(dfutil0.bobinas)]     
        a = df_pedidos.value_counts('formato')
        pedidos = dict(sorted(a.items()))
        cortes = list(pedidos.keys())
        bobinas = list(pedidos.values())
        
        return cortes, pedidos, bobinas, dfutil0  
    
def combinaciones(cortes, pedidos, val_min): 
        bob = []
        num = []
        val_max = 383

        for i in (range(2,len(cortes)+1)):
            bob.append(cortes[:i])
            num.append(i-1)
        bob.reverse()
        dict_comb = dict(zip(num,bob))
        

        posibles = []
        for u in dict_comb:
            for i in dict_comb[u]:
                if val_min <= max(dict_comb[u]) + i <= val_max:
                    a = (max(dict_comb[u]),i)
                    posibles.append(a)
                for o in pedidos:
                    if val_min <= max(dict_comb[u]) + o + i <= val_max and i <= o:
                        b = (max(dict_comb[u]),i,o)
                        posibles.append(b)
                    
        minimo = cortes[0]
        if val_min <= minimo * 3 <= val_max:
            posibles.append((minimo,minimo,minimo))
            
        if val_min <= minimo * 4 <= val_max:
            posibles.append((minimo, minimo, minimo, minimo))
            
        paysan = []
        a = [ ]
        for j in cortes: 
            for i in posibles:       
                a.append(i.count(j))
                
        b = []        
        for i in range(0, len(a), len(posibles)):
            b.append(a[i:i+ len(posibles)])
            
        for i in posibles:
            if sum(i)<370:
                paysan.append(383-sum(i))
            else:
                paysan.append(0)  
                
        dic_pulp = dict(zip(range(len(cortes)),b))
        dic_pulp['posibles'] = posibles
        dic_pulp['banda'] = paysan
        df = pd.DataFrame(dic_pulp)
        
        return posibles, paysan, df
    
def solmatriz(df,cortes,posibles,bobinas, d, peso, E, pedidos, dfutil0):
    m = sum(d)
    clientes = []
    for i in range(len(cortes)):
        a = str(i)
        clientes.append(a)
    

    minimos = ["minimouno","minimodos","minimotres","minimocuatro","minimocinco","minimoseis","minimosiete","minimoocho","minimonueve", "minimodiez","minimoonce","minimodoce","minimotrece","minimocatorce","minimoquince","minimodieciseis","minimodiecisiete","minimodieciocho","minimodiecinueve","minimoveinte","minimoveintiuno","minimoveintidos","minimoveintitres",
               "minimoveinticuatro","minimoveinticinco","minimoveintiseis","minimoveintisiete","minimoveintiocho","minimoveintinueve","minimotreinta","minimotreintaiuno","minimotreintaidos","minimotreintaitres","minimotreintaicuatro","minimotreintaicinco","minimotreintaiseis","minimotreintaisiete","minimotreintaiocho","minimotreintainueve","minimocuarenta","minimocuarentaiuno","minimocuarentaidos","minimocuarentaitres",
               "minimocuarentaicuatro","minimocuarentaicinco","minimocuarentaiseis","minimocuarentaisiete","minimocuarentaiocho","minimocuarentainueve","minimocincuenta"]
    maximos = ["maximouno","maximodos","maximotres","maximocuatro","maximocinco","maximoseis","maximosiete","maximoocho","maximonueve","maximodiez","maximoonce","maximodoce","maximotrece","maximocatorce","maximoquince","maximodieciseis","maximodiecisiete","maximodieciocho","maximodiecinueve","maximoveinte","maximoveintinuo","maximoveintidos",
               "maximoveintitre","maximoveinticuatro","maximoveinticinco","maximoveintiseis","maximoveintisiete","maximoveintiocho","maximoveintinueve","maximotreinta","maximotreintaiuno","maximotreintaidos","maximotreintaitres","maximotreintaicuatro","maximotreintaicinco","maximotreintaiseis","maximotreintaisieta","maximotreintaiocho","maximotreintainueve","maximocuarenta","maximocuarentaiuno","maximocuarentaidos",
               "maximocuarentaitres","maximocuarentaicuatro","maximocuarentaicinco","maximocuarentaiseis","maximocuarentisiete","maximocuarentaiocho","maximocuarentainueve","maximocincuenta"]   
    problema = LpProblem("Problemacombinacion",LpMinimize)
    posibles = list(df['posibles'])
    banda = dict(zip(posibles, df['banda']))

    posibles_vars = LpVariable.dicts("posibles", posibles, 0, cat='Integer')

    problema += lpSum([banda[i]*posibles_vars[i] for i in posibles]), "banda total de la produccion"

    for i in range(len(cortes)):       
        clientes[i] = dict(zip(posibles,df[i]))
        problema += lpSum([clientes[i][f] * posibles_vars[f] for f in posibles]) >= bobinas[i], minimos[i]
        problema += lpSum([clientes[i][f] * posibles_vars[f] for f in posibles]) <= bobinas[i]+E[i], maximos[i]

## Interpretacion y resolucion de la matriz a partir de las restricciones del problema
    resultado  = {}
    problema.writeLP("SimplecombProblem.lp")
    problema.solve()
    resultado = []
    cantidad = []
    print("Status:", LpStatus[problema.status])
    print("La combinacion balanceada óptima(menor banda), donde cada unidad equivale a 1 comb, consiste de\n"+"-"*110)
    for v in problema.variables():
        if v.varValue>0:
            resultado.append(v.name)
            cantidad.append(int(v.varValue))
    peso_banda = (value(problema.objective))*peso/2/383
    kilos = 'la cantidad de banda producida es :' ,round(peso_banda,2), 'kg'
    kilos_st = str(kilos)
    n = (peso_banda * 100 ) / m
    print("El total de kg producidos : ", round(m+peso_banda,2) , "kg")
    print("El total de banda producida es de: ", round(peso_banda,2), "kg")
    print("El porcentaje de banda producida es de: ", round(n,2), "%")
## convierto de archivo lp a una lista de int para poder manipular los datos introduciendo la pasta a cada corte
## luego los convierto a str para exportar a excel
    povarval = []
    povarkeys = []
    for i in posibles_vars:
        povarval.append(str(posibles_vars[i]))
        povarkeys.append(i)
    dic = dict(zip(povarkeys,povarval))

    claves = []
    def clave(dic, value):
        return [key for key, val in dic.items() if val == value]
    for i in resultado:  
        keys = clave(dic, i)
        claves.append(keys)
    pastas = []
    for i in range(len(claves)):
        if sum(claves[i][0])<370:
            pastas.append(383-sum(claves[i][0]))
        else:
            pastas.append(0)  
    a = []
    for i in range(len(claves)):
        a.append(list(claves[i][0]))
    for i in range(len(claves)):
        claves[i].append(pastas[i])
    lista = []
    for i in range(len(claves)):
        a = claves[i][0]
        b = list(a)
        lista.append(b)
    pastas
    for i in range(len(claves)):
        lista[i].append(pastas[i])
    cantidad1 = []
    for i in range((len(cantidad))):
        cantidad1.append(cantidad[i]*3500)
    #for i in range(len(cantidad1)):
        #print(lista[i], "=", cantidad1[i])
    lista4 = []
    for i in lista:
        a = str(i)
        lista4.append(a)
    diccionariofinal = {"Cortes":lista4, "cantidad":cantidad1, "bajadas":cantidad}
    programa = pd.DataFrame(diccionariofinal)
    
    dic_solucion = dict(zip(lista4,cantidad))
    sumvariables = []
    listafinal = []
    totales = []
    for i in range(len(lista)):
        for o in range(len(lista[i])):
            for j in range(cantidad[i]):
                totales.append(lista[i][o])
    df = {'prod':totales}
    df1 = pd.DataFrame(df)
    total_producido = df1['prod'].value_counts()
    ## se ordena el programa segun ancho maximo de las bajadas para no tener que estar modificando cosntantemente el ancho en maquina
    lista
    ancho_maximo = []
    for i in lista:
        a = sum(i)
        ancho_maximo.append(a)
    programa['ancho'] = ancho_maximo
    programa = programa.sort_values('ancho')

    total_formatos = list(total_producido.index)
    total_cantidad = list(total_producido.values)
    total = dict(zip(total_formatos,total_cantidad))
    dict1 = total
    dict2 = pedidos
    dict3 = defaultdict(list)
    for k, v in chain(dict1.items(), dict2.items()):
        dict3[k].append(v)

        valores = list(dict3.values())
        claves = list(dict3.keys())
    k = []
    j = []
    l = []
    n = []
    for i in range(len(valores)):
      if len(valores[i])>1:
        k.append(valores[i][0])
        l.append(valores[i][1])
        j.append(claves[i])
        n.append(valores[i][0]-valores[i][1])

    diccionario_pedids = {'corte':j,'pedidos':l,'fabricado':k,'complemento':n}
    df_ped = pd.DataFrame(diccionario_pedids)

    for i in range(len(df_ped['complemento'])):
      if df_ped['complemento'][i] != 0:
        dfutil0=dfutil0.append({'IV' : 'COMPLEMENTO' , 'formato' : df_ped['corte'][i], 'bobinas' : df_ped['complemento'][i]} , ignore_index=True)
    diccionario = {'lista':lista, 'cantidad':cantidad}
    programanuevo = pd.DataFrame(diccionario)
    programa1 = programanuevo.loc[programanuevo.index.repeat(programanuevo.cantidad)]     
    programa2 = programa1.reset_index(drop=True)
    dfutil1 = dfutil0.loc[dfutil0.index.repeat(dfutil0.bobinas)]  
    dfutil2 = dfutil1.reset_index(drop=True)

    nueva_lista = []
    dfutil3 = dfutil2
    for x in programa2['lista']:
      a = x[:-1]
      nueva_lista.append(a)
    lista_final = []
    for x in nueva_lista:
      lista_temp = []
      lista_final.append(lista_temp)
      for i in x:
        a = 0    
        while dfutil3['formato'][a] != i:
          a = a + 1
        else:
         lista_temp.append(str(dfutil3['formato'][a])+' ('+'IV: '+str(dfutil3['IV'][a])+')')
         dfutil3 = dfutil3.drop(a).reset_index(drop=True)
     
    lista_str = []
    for x in lista_final:
      lista_str.append(str(x))
    a = {'lista':lista_str}
    b = pd.DataFrame(a)
    d = []
    programa_iv = b.value_counts()
    cortes_prog = list(programa_iv.index)
    bajadas_prog = list(programa_iv.values)
    dic4 = {'corte':cortes_prog,'bajadas':bajadas_prog}
    df4 = pd.DataFrame(dic4)
    return(programa, total, lista, cantidad, df4, df_ped )


     |████████████████████████████████| 14.2 MB 1.6 MB/s 


In [ ]:
#@title Importacion de datos: los siguientes datos son los formatos en orden ascendente
## ingreso de variables importantes: cantidad de pedidos, valor minimo, valor maximo,
## planilla excel(clientes, columna FTO y columna PED)           
## datos del archivo excel.
File = pd.read_excel('pedidos.xlsx')
data_base = pd.DataFrame(File)
#ordenes = len(data_base['IV'])
ordenes = 300
#columna de formatos
b = File["FTO"][:ordenes]
#columna de kg
d = File["SALDO"][:ordenes]
#columna de clientes
cliente = File["IV"][:ordenes]
##aca van la cantidad de filas del excel:
val_max = 383
## el error a aplicar se lo debe modificar en orden de menor a mayor para cada uno de los formatos.
peso = 7000    
cortes, pedidos, bobinas, dfutil0 = ordenamiento(ordenes, peso, b, d, cliente)
print(pedidos)

{80: 4, 85: 4, 90: 8, 105: 59, 111: 5, 115: 7, 120: 4, 125: 37, 145: 8, 150: 19, 155: 16, 160: 47, 165: 14, 170: 3, 180: 71, 188: 16, 190: 50, 200: 99, 205: 4, 213: 27, 215: 10, 220: 16, 225: 6, 226: 17, 230: 24, 235: 5, 245: 11, 255: 7}


### Ingresar la cantidad de bobinas de mas permitidas para cada formato en forma ascendente

In [ ]:
E = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


### Maximo de banda permitido por bajada

In [ ]:
Banda = 33

In [ ]:
#@title Resolucion
val_min = 383-Banda
posibles, paysan, df = combinaciones(cortes, pedidos, val_min)
programa, total_producido, lista, cantidad, df4, df_ped = solmatriz(df,cortes,posibles,bobinas,d, peso, E, pedidos, dfutil0)

data_null = {}
df_null = pd.DataFrame(data_null)
df_null.to_excel('Reporte.xlsx', index = False) 
writer = pd.ExcelWriter('Reporte.xlsx')
dfutil0.to_excel(writer, 'Informe pedidos', index = False)
programa.to_excel(writer, 'OP con banda', index = False)
df4.to_excel(writer, 'OP con IV', index = False)
df_ped.to_excel(writer, 'Informe de Produccion', index = False)
writer.save()
writer.close()


Status: Optimal
La combinacion balanceada óptima(menor banda), donde cada unidad equivale a 1 comb, consiste de
--------------------------------------------------------------------------------------------------------------
El total de kg producidos :  984536.55 kg
El total de banda producida es de:  21036.55 kg
El porcentaje de banda producida es de:  2.18 %
